# UK Number 1 Songs of 1999

1999 was the last big 'hurrah' for the music industry during the CD era before the rise of the internet, file-sharing and illegal downloads through sites such as Napster decimated record sales. It was only once downloads became recognised as an official method of purchasing music, and streaming services such as Spotify really took hold, that sales for singles began to turn around (album sales, on the hand, still have a long way to go).

For this project, the aim is to scrape data in a Wikitable about the songs that topped the UK Singles Chart each week of 1999 and the respective sales accumulated. Once that's done, we will transform the data so that it is ready to be loaded into a PostgreSQL database managed by AWS.

In [0]:
# Import libraries 
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *

In [0]:
# Request permission from the Wikipedia page to fetch data
url = "https://en.wikipedia.org/wiki/1999_in_British_music_charts#Year-end_charts"
response = requests.get(url)

# Verify that permission has been granted - the response code returned should be 200 if so
response.status_code

Out[3]: 200

In [0]:
# Print user-friendly code representing all the data from the Wikipedia page
soup = BeautifulSoup(response.text)
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-disabled vector-feature-client-preferences-disabled" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   1999 in British music charts - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-l

## 1. Extract the data

In the original Wikipedia table, song/artist pairs that stayed at Number 1 for more than 1 week are listed once only, so we need to 'fill in the gaps' to ensure that the dataframe we want to create correctly labels a particular sales week with the right song/artist pair.  

In the list created below, we can group items like so - all information relating to a particular week starts with an item containing the last date of the chart week, followed by the name of the song and its artist(s), and ends with an item containing the number of sales accumulated. 

So that we don't get ahead of ourselves, let's test our approach on a couple of examples within our list relating to songs that spent more than 1 week at Number 1.

In [0]:
# Create a list containing scraped data about each Number 1 song in 1999    
data = [td.text for td in soup.find('table', class_='wikitable plainrowheaders').tbody.findAll('td')]
data

Out[5]: ['2 January',
 '"Chocolate Salty Balls"',
 'Chef',
 '320,000\n',
 '9 January',
 '"Heartbeat" / "Tragedy"',
 'Steps',
 '98,000\n',
 '16 January',
 '"Praise You"',
 'Fatboy Slim',
 '80,913\n',
 '23 January',
 '"A Little Bit More"',
 '911',
 '75,400\n',
 '30 January',
 '"Pretty Fly (for a White Guy)"',
 'The Offspring',
 '140,000\n',
 '6 February',
 '"You Don\'t Know Me"',
 'Armand Van Helden featuring Duane Harden',
 '118,500\n',
 '13 February',
 '"Maria"',
 'Blondie',
 '128,000\n',
 '20 February',
 '"Fly Away"',
 'Lenny Kravitz',
 '123,000\n',
 '27 February',
 '"...Baby One More Time"',
 'Britney Spears',
 '463,722\n',
 '6 March',
 '231,000\n',
 '13 March',
 '"When the Going Gets Tough"',
 'Boyzone',
 '213,000\n',
 '20 March',
 '197,000\n',
 '27 March',
 '"Blame It on the Weatherman"',
 'B*Witched',
 '90,000\n',
 '3 April',
 '"Flat Beat"',
 'Mr. Oizo',
 '283,000\n',
 '10 April',
 '184,000\n',
 '17 April',
 '"Perfect Moment"',
 'Martine McCutcheon',
 '200,000\n',
 '24 April',
 '1

In [0]:
# "...Baby One More Time" by Britney Spears 
britney = ['27 February', 
           '"...Baby One More Time"', 
           'Britney Spears', 
           '463,722\n', 
           '6 March', 
           '231,000\n']

# Any item relating to the chart week contains a month of the year
# Any item relating to the number of sales that week contains the special character '\n'
# We can use this information to iterate through the following loop in order to fill in missing data

months = ['January', 
          'February', 
          'March', 
          'April', 
          'May', 
          'June', 
          'July', 
          'August', 
          'September', 
          'October', 
          'November', 
          'December']

# Iterate through the available indices of the list
for i in range(len(britney)):
    
    # Print the current item of the list
    print(f"Check the following item: {britney[i]}")
    
    # Check if the current item is the chart week (i.e. contains a month of the year)
    for m in months:
        # If so, assign a new variable to the given month and break the inner for loop
        if m in britney[i]:
            month = m
            break
            
    # Move the outer for loop along to the next item if the current item is the chart week        
    if month in britney[i]:
        continue
        
    # Check if the current item is NOT the number of sales
    if '\n' not in britney[i]:
        # If so and the next item also isn't the number of sales, 
        # prepare variables for the song and artist before continuing the loop to the next item
        if '\n' not in britney[i+1]:
            song = britney[i]
            artist = britney[i+1]
            continue
        # If so and the next item IS the number of sales, 
        # continue the loop to the next item without creating new variables
        else:
            continue       
    else:
        # If the current item IS the number of sales, 
        # and none of the previous 2 items relate to the chart week, continue the loop to the next item
        if (month not in britney[i-1]) and (month not in britney[i-2]):
            continue
        # If at least one of the 2 previous items is the chart week, insert the song and artist names into the list
        else:
            britney.insert(i, song)
            britney.insert(i+1, artist)
            continue
            
britney           

Check the following item: 27 February
Check the following item: "...Baby One More Time"
Check the following item: Britney Spears
Check the following item: 463,722

Check the following item: 6 March
Check the following item: 231,000

Out[6]: ['27 February',
 '"...Baby One More Time"',
 'Britney Spears',
 '463,722\n',
 '6 March',
 '"...Baby One More Time"',
 'Britney Spears',
 '231,000\n']

In [0]:
# "Livin' la Vida Loca" by Ricky Martin 
ricky = ['17 July', 
         '"Livin\' la Vida Loca"', 
         'Ricky Martin', 
         '131,000\n', 
         '24 July', 
         '125,000\n', 
         '31 July', 
         '96,600\n']

# Iterate through the available indices of the list
for i in range(len(ricky)):
    
    # Print the current item of the list
    print(f"Check the following item: {ricky[i]}")
    
    # Check if the current item is the chart week (i.e. contains a month of the year)
    for m in months:
        # If so, assign a new variable to the given month and break the inner for loop
        if m in ricky[i]:
            month = m
            break
            
    # Move the outer for loop along to the next item if the current item is the chart week
    if month in ricky[i]: 
        print(ricky[i])
        continue
        
    # Check if the current element is NOT the number of sales
    if '\n' not in ricky[i]:
        # If so and the next item also isn't the number of sales, 
        # prepare variables for the song and artist before continuing the loop to the next item
        if '\n' not in ricky[i+1]:
            song = ricky[i]
            artist = ricky[i+1]
            print(ricky[i])
            continue
        # If so and the next item IS the number of sales, 
        # continue the loop to the next item without creating new variables
        else:
            print(ricky[i])
            continue
    else:
        # If the current item IS the number of sales, 
        # and none of the previous 2 items relate to the chart week, continue the loop to the next item
        if (month not in ricky[i-1]) and (month not in ricky[i-2]):
            print(ricky[i])
            continue
        # If at least one of the 2 previous items is the chart week, insert the song and artist names into the list
        else:
            ricky.insert(i, song)
            ricky.insert(i+1, artist)
            print(ricky[i])
            
ricky

Check the following item: 17 July
17 July
Check the following item: "Livin' la Vida Loca"
"Livin' la Vida Loca"
Check the following item: Ricky Martin
Ricky Martin
Check the following item: 131,000

131,000

Check the following item: 24 July
24 July
Check the following item: 125,000

"Livin' la Vida Loca"
Check the following item: Ricky Martin
Ricky Martin
Check the following item: 125,000

125,000

Out[7]: ['17 July',
 '"Livin\' la Vida Loca"',
 'Ricky Martin',
 '131,000\n',
 '24 July',
 '"Livin\' la Vida Loca"',
 'Ricky Martin',
 '125,000\n',
 '31 July',
 '96,600\n']

We have a problem in the last example - "Ricky Martin" and "Livin' La Vida Loca" wasn't added to the list in relation to its last week at Number 1 (week ending 31st July). 

Why is this happening? Notice that the outer loop doesn't iterate through the original list but through its available indices. Since the loop aims to update the list, this method may not account for this or multiple occurences of the same item (for the case of the song and artist). 

Let's try again but this time iterate through the actual list of items instead. 

In [0]:
ricky = ['17 July', '"Livin\' la Vida Loca"', 'Ricky Martin', '131,000\n', '24 July', '125,000\n', '31 July', '96,600\n']

# Iterate through the actual items of the list
for r in ricky:
    
    # Print out the current item in the list
    print(f"Check the following item: {r}")
    
    # Find the index of the last occurence of the current item in the list
    index = len(ricky) - [i for i in reversed(ricky)].index(r) - 1 
    
    # Check if the current item is the chart week 
    for m in months: # check if any of the months are contained within an element of the main list
        if m in r:
            month = m
            break
    if month in r: 
        continue
    # Check if the current element is NOT the number of sales
    if '\n' not in r:
        # If so and the next element also isn't the number of sales, 
        # prepare variables for the song and artist before continuing the loop to the next element
        if '\n' not in ricky[index + 1]:
            song = r
            artist = ricky[index + 1]
            continue
        # If so and the next element IS the number of sales, 
        # continue the loop to the next element without creating new variables
        else:
            continue
    else:
        # If the current element IS the number of sales, 
        # and none of the previous 2 elements relate to the chart week, continue the loop to the next element
        if (month not in ricky[index - 1]) and (month not in ricky[index - 2]):
            continue
        # If at least one of the 2 previous elements is the chart week, insert the song and artist names into the list
        else:
            ricky.insert(index, song)
            ricky.insert(index + 1, artist)
            
ricky

Check the following item: 17 July
Check the following item: "Livin' la Vida Loca"
Check the following item: Ricky Martin
Check the following item: 131,000

Check the following item: 24 July
Check the following item: 125,000

Check the following item: Ricky Martin
Check the following item: 125,000

Check the following item: 31 July
Check the following item: 96,600

Check the following item: Ricky Martin
Check the following item: 96,600

Out[8]: ['17 July',
 '"Livin\' la Vida Loca"',
 'Ricky Martin',
 '131,000\n',
 '24 July',
 '"Livin\' la Vida Loca"',
 'Ricky Martin',
 '125,000\n',
 '31 July',
 '"Livin\' la Vida Loca"',
 'Ricky Martin',
 '96,600\n']

Now that this new program works on these examples, let's apply it to the entire list of scraped data. If successful, the updated list should return 208 items - 4 items (chart week, song, artist(s), sales) for each week of the year.

In [0]:
data = [td.text for td in soup.find('table', class_='wikitable plainrowheaders').tbody.findAll('td')]
months = ['January', 
          'February', 
          'March', 
          'April', 
          'May', 
          'June', 
          'July', 
          'August', 
          'September', 
          'October', 
          'November', 
          'December']

for d in data:
    index = len(data) - [i for i in reversed(data)].index(d) - 1
    for m in months: 
        if m in d:
            month = m
            break
    if month in d: 
        continue  
    if '\n' not in d:
        if '\n' not in data[index + 1]:
            song = d
            artist = data[index + 1]
            continue
        else:
            continue
    else:
        if (month not in data[index - 1]) and (month not in data[index - 2]):
            continue
        else:
            data.insert(index, song)
            data.insert(index + 1, artist)
            
print(f"There are {len(data)} items in the updated list")            
data

There are 206 items in the updated list
Out[9]: ['2 January',
 '"Chocolate Salty Balls"',
 'Chef',
 '320,000\n',
 '9 January',
 '"Heartbeat" / "Tragedy"',
 'Steps',
 '98,000\n',
 '16 January',
 '"Praise You"',
 'Fatboy Slim',
 '80,913\n',
 '23 January',
 '"A Little Bit More"',
 '911',
 '75,400\n',
 '30 January',
 '"Pretty Fly (for a White Guy)"',
 'The Offspring',
 '140,000\n',
 '6 February',
 '"You Don\'t Know Me"',
 'Armand Van Helden featuring Duane Harden',
 '118,500\n',
 '13 February',
 '"Maria"',
 'Blondie',
 '128,000\n',
 '20 February',
 '"Fly Away"',
 'Lenny Kravitz',
 '123,000\n',
 '27 February',
 '"...Baby One More Time"',
 'Britney Spears',
 '463,722\n',
 '6 March',
 '"...Baby One More Time"',
 'Britney Spears',
 '231,000\n',
 '13 March',
 '"When the Going Gets Tough"',
 'Boyzone',
 '213,000\n',
 '20 March',
 '"When the Going Gets Tough"',
 'Boyzone',
 '197,000\n',
 '27 March',
 '"Blame It on the Weatherman"',
 'B*Witched',
 '90,000\n',
 '3 April',
 '"Flat Beat"',
 'Mr. Oizo

Almost there but not quite right! The list returned 206 items, so we are missing 2 items.

Upon further investigation, the issue relates to the song "Sweet Like Chocolate" by Shanks & Bigfoot. Let's take a closer a look to see what's going wrong.

In [0]:
shanks = ['29 May',
 '"Sweet Like Chocolate"',
 'Shanks & Bigfoot',
 '251,000\n',
 '5 June',
 '141,000\n']

for d in shanks:
    print(f"Check the following item: {d}")
    index = len(shanks) - [i for i in reversed(shanks)].index(d) - 1
    for m in months:
        if m in d:
            month = m
            break
    if month in d: 
        continue
    if '\n' not in d:
        if '\n' not in shanks[index + 1]:
            song = d
            artist = shanks[index + 1]
            continue
        else:
            continue
    else:
        if (month not in shanks[index - 1]) and (month not in shanks[index - 2]):
            continue
        else:
            shanks.insert(index, song)
            shanks.insert(index + 1, artist)
            
shanks

Check the following item: 29 May
Check the following item: "Sweet Like Chocolate"
Check the following item: Shanks & Bigfoot
Check the following item: 251,000

Check the following item: 5 June
Check the following item: 141,000

Check the following item: Shanks & Bigfoot
Check the following item: 141,000

Out[10]: ['29 May',
 '"Sweet Like Chocolate"',
 'Shanks & Bigfoot',
 '251,000\n',
 '5 June',
 '"Sweet Like Chocolate"',
 'Shanks & Bigfoot',
 '141,000\n']

One solution is to insert the correct missing data into the list. Note that this only works for this specific case - a more general solution would have to be applied in case we wanted to scrape data from similar Wikipedia tables and came across similar issues. Once a more general solution is found, this notebook will be updated. 

In [0]:
# Find the index of '5 June' within the list of scraped data
shanks_date = data.index("5 June")

# Insert 'Sweet Like Chocolate' and 'Shanks & Bigfoot' into the list to complete the dataset
data.insert(shanks_date + 1, "Shanks & Bigfoot")
data.insert(shanks_date + 1, "Sweet Like Chocolate")

print(f"There are {len(data)} items in the updated list")            
data

There are 208 items in the updated list
Out[11]: ['2 January',
 '"Chocolate Salty Balls"',
 'Chef',
 '320,000\n',
 '9 January',
 '"Heartbeat" / "Tragedy"',
 'Steps',
 '98,000\n',
 '16 January',
 '"Praise You"',
 'Fatboy Slim',
 '80,913\n',
 '23 January',
 '"A Little Bit More"',
 '911',
 '75,400\n',
 '30 January',
 '"Pretty Fly (for a White Guy)"',
 'The Offspring',
 '140,000\n',
 '6 February',
 '"You Don\'t Know Me"',
 'Armand Van Helden featuring Duane Harden',
 '118,500\n',
 '13 February',
 '"Maria"',
 'Blondie',
 '128,000\n',
 '20 February',
 '"Fly Away"',
 'Lenny Kravitz',
 '123,000\n',
 '27 February',
 '"...Baby One More Time"',
 'Britney Spears',
 '463,722\n',
 '6 March',
 '"...Baby One More Time"',
 'Britney Spears',
 '231,000\n',
 '13 March',
 '"When the Going Gets Tough"',
 'Boyzone',
 '213,000\n',
 '20 March',
 '"When the Going Gets Tough"',
 'Boyzone',
 '197,000\n',
 '27 March',
 '"Blame It on the Weatherman"',
 'B*Witched',
 '90,000\n',
 '3 April',
 '"Flat Beat"',
 'Mr. Oiz

Perfect! We now have our complete dataset to create the final dataframe.

In [0]:
# Remove special characters from Sales data so that they only include digits
for i in range(3,208,4):
    data[i] = data[i].strip("\n").replace(",", "")

# Remove extra quotation marks from Song data
for i in range(1,206,4):
    data[i] = data[i].replace('"', "")

# Convert the extracted data into a list of tuples, each representing the rows that will form the subsequent dataframe
it = iter(data)
rows = list(zip(it, it, it, it))
rows

Out[12]: [('2 January', 'Chocolate Salty Balls', 'Chef', '320000'),
 ('9 January', 'Heartbeat / Tragedy', 'Steps', '98000'),
 ('16 January', 'Praise You', 'Fatboy Slim', '80913'),
 ('23 January', 'A Little Bit More', '911', '75400'),
 ('30 January', 'Pretty Fly (for a White Guy)', 'The Offspring', '140000'),
 ('6 February',
  "You Don't Know Me",
  'Armand Van Helden featuring Duane Harden',
  '118500'),
 ('13 February', 'Maria', 'Blondie', '128000'),
 ('20 February', 'Fly Away', 'Lenny Kravitz', '123000'),
 ('27 February', '...Baby One More Time', 'Britney Spears', '463722'),
 ('6 March', '...Baby One More Time', 'Britney Spears', '231000'),
 ('13 March', 'When the Going Gets Tough', 'Boyzone', '213000'),
 ('20 March', 'When the Going Gets Tough', 'Boyzone', '197000'),
 ('27 March', 'Blame It on the Weatherman', 'B*Witched', '90000'),
 ('3 April', 'Flat Beat', 'Mr. Oizo', '283000'),
 ('10 April', 'Flat Beat', 'Mr. Oizo', '184000'),
 ('17 April', 'Perfect Moment', 'Martine McCutcheon',

In [0]:
# Initialise Spark session to create Spark dataframe
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()
columns = ["ChartDate_WeekEnding", "Song", "Artists", "Sales"]
df = spark.createDataFrame(data = rows, schema = columns)

# Display dataframe
display(df)

ChartDate_WeekEnding,Song,Artists,Sales
2 January,Chocolate Salty Balls,Chef,320000
9 January,Heartbeat / Tragedy,Steps,98000
16 January,Praise You,Fatboy Slim,80913
23 January,A Little Bit More,911,75400
30 January,Pretty Fly (for a White Guy),The Offspring,140000
6 February,You Don't Know Me,Armand Van Helden featuring Duane Harden,118500
13 February,Maria,Blondie,128000
20 February,Fly Away,Lenny Kravitz,123000
27 February,...Baby One More Time,Britney Spears,463722
6 March,...Baby One More Time,Britney Spears,231000


## 2. Transform the data

In [0]:
# Check the data type for each column
df.printSchema()

root
 |-- ChartDate_WeekEnding: string (nullable = true)
 |-- Song: string (nullable = true)
 |-- Artists: string (nullable = true)
 |-- Sales: string (nullable = true)



As we can see from the dataframe's schema, we need to transform some columns. The Sales column should contain integer values only, and the Chart Date column should contain dates only. 

In [0]:
# Create a UDF (user-defined function) to transform data in the Chart Date column
def make_date(date):

    # Isolate the day from the date argument, ensuring that it always contains 2 digits
    if len(date.split()[0]) == 1:
        day = "0" + date.split()[0]
    else:
        day = date.split()[0]

    # Create a dictionary linking months to their associated numbers
    keys = ['January', 
          'February', 
          'March', 
          'April', 
          'May', 
          'June', 
          'July', 
          'August', 
          'September', 
          'October', 
          'November', 
          'December']
    values = ["01","02","03","04","05","06","07","08","09","10","11","12"]
    month_dict = dict(map(lambda x,y: (x,y), keys,values))
    
    # Based on this dictionary, isolate the month from the date argument in numbered form  
    for m in keys:
        if m == date.split()[1]:
            month = month_dict[m]

    # Now convert the date string argument to a datetime.date() object in desired format
    return datetime.strptime("1999" + "-" + month + "-" + day,"%Y-%m-%d")

# Finalise UDF, specifying the data type it should return
makeDateUDF = udf(lambda x: make_date(x), DateType())

# Now apply UDF to Chart Date column, and convert Sales column from strings to integers 
df = df.withColumn('ChartDate_WeekEnding', makeDateUDF(col('ChartDate_WeekEnding'))).withColumn('Sales', col('Sales').cast('integer'))

# Finally, make sure the dataframe is sorted in chronological order before it gets loaded into the database later to avoid any potential discrepancies
df = df.sort(df.ChartDate_WeekEnding.asc())

# View transformed dataframe
display(df)

ChartDate_WeekEnding,Song,Artists,Sales
1999-01-02,Chocolate Salty Balls,Chef,320000
1999-01-09,Heartbeat / Tragedy,Steps,98000
1999-01-16,Praise You,Fatboy Slim,80913
1999-01-23,A Little Bit More,911,75400
1999-01-30,Pretty Fly (for a White Guy),The Offspring,140000
1999-02-06,You Don't Know Me,Armand Van Helden featuring Duane Harden,118500
1999-02-13,Maria,Blondie,128000
1999-02-20,Fly Away,Lenny Kravitz,123000
1999-02-27,...Baby One More Time,Britney Spears,463722
1999-03-06,...Baby One More Time,Britney Spears,231000


In [0]:
# Check new dataframe schema for updated data types
df.printSchema()

root
 |-- ChartDate_WeekEnding: date (nullable = true)
 |-- Song: string (nullable = true)
 |-- Artists: string (nullable = true)
 |-- Sales: integer (nullable = true)



## 3. Create the database
<br>
<ol>
  <li>Create a free-tier AWS account <a href='https://aws.amazon.com/free/'>here</a> if you don't already have one, and head to the RDS Management Console.</li>
  <li>Go to <i>Parameter groups</i> and create a custom parameter group for the database group family <i>postgres15</i>.
  <br>
  Then modify the parameters below with the following values:
    <ul>
      <li><i>rds.force_ssl</i> = 0</li>
      <li><i>rds.logical_replication</i>  = 1</li>
      <li><i>wal_sender_timeout</i> = 0</li>
      <li><i>max_worker_processes</i> = 12</li>
      <li><i>max_logical_replication_workers</i> = 1</li>
      <li><i>autovacuum_max_workers</i> = 3</li>
      <li><i>max_parallel_workers</i> = 8</li>
      <li><i>shared_preload_libraries</i> = pglogical</li>
    </ul>
  </li>
  <li>Go to <i>Databases</i> and create a free-tier PostgreSQL database running on the engine version <i>PostgreSQL 15</i>, 
  <br>
  following the instructions <a href='https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/CHAP_Tutorials.WebServerDB.CreateDBInstance.html'>here</a> except:
  <ul>
      <li>In the <i>Connectivity</i> section, select <i>Don’t connect to an EC2 compute resource</i>.</li>
      <li>In the same section, ensure the default VPC, DB subnet group and VPC security group options are selected.</li>
      <li>In the <i>Additional configuration</i> section, leave the initial database name blank - it defaults to <i>postgres</i>.</li>
      <li>In the same section, select the DB parameter group you've just created.</li>
    </ul>
  <li>Once created, under the database's <i>Connectivity & security</i> go to <i>Security</i> and click the link under <i>VPC security groups.</i>
  <br>
  Go to <i>Inbound rules</i> and edit the security group's inbound rules so that they only include the follow
    <ul>
      <li><i>Type</i>: All traffic, <i>Source type</i>: Custom, <i>Source</i>: ::/0</li>
      <li><i>Type</i>: All traffic, <i>Source type</i>: Custom, <i>Source</i>: 0.0.0.0/0</li>
      <li><i>Type</i>: PostgreSQL, <i>Source type</i>: Custom, <i>Source</i>: ::/0</li>
      <li><i>Type</i>: PostgreSQL, <i>Source type</i>: Custom, <i>Source</i>: 0.0.0.0/0</li>
    </ul>
  </li>
  <li>Download and install PostgreSQL by following <a href='https://www.postgresqltutorial.com/postgresql-getting-started/install-postgresql/'>this guide</a>.</li>
  <li>Download and set up the PostgreSQL driver <a href='https://jdbc.postgresql.org/documentation/setup/'>here</a>.</li>
  <li>Open pgAdmin on your desktop and connect to the database just created by following <a href='https://www.postgresqltutorial.com/postgresql-getting-started/connect-to-postgresql-database/'>these instructions</a>, 
  <br>
  using the database's endpoint (found in RDS) as the hostname and keeping <i>postgres</i> as the maintanence database.</li>
</ol>

## 4. Load the data into the database

In [0]:
# Prepare credentials to connect to the database and the table where the data will be stored
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://database-1.chadxmjk4y7a.eu-north-1.rds.amazonaws.com/" #"jdbc:postgresql://{insert database endpoint url}/"
table = "mwag_pyspark.songsales" #"{choose schema name}.{choose table name}"
user = "postgres" #"{insert database username}"
password = "postgres123" #"{insert database password}"

# Write data to the database
df.write.format("jdbc").option("driver", driver).option("url", url).option("dbtable", table).option("mode", "append").option("user",user).option("password", password).save()

All done! Now that we've completed this ETL pipeline, we can move on to the next project where we will build a CDC pipeline to migrate the data from our PostgreSQL database to a new target MySQL database, and replicate any changes we've made to the source data in the target. Click <a href='#'>here</a> to get started.